In [1]:
import numpy as np
from numpy.linalg import eig
from numpy import mean
from scipy import stats
from math import factorial

def variance(x):
    mean = np.mean(x)
    total = 0
    for element in x:
        total += (element - mean)**2
    return total/(len(x)-1)

def std(x):
    return np.sqrt(variance(x))

def skew(x):
    var = variance(x)
    mean = np.mean(x)
    total = 0
    for element in x:
        total += (element - mean)**3
    return total / (len(x) * (var**(3/2)))

def covariance(x, y):
    mean_x, mean_y = np.mean(x), np.mean(y)
    total = 0
    for i in range(len(x)):
        total += (x[i] - mean_x)*(y[i] - mean_y)
    
    return total/(len(x)-1)

def spearman(x, y):
    """Calculates Spearman Rank Coefficient for two arrays of data,
    x and y."""
    sorted_x_idx = stats.rankdata(x)
    sorted_y_idx = stats.rankdata(y)
    total = 0
    N = len(x)
    for i in range(N):
        total += (sorted_x_idx[i] - sorted_y_idx[i])**2
        
    return (1 - (6*total) / (N * (N**2 - 1)))

def binomial(r, p, n):
    frac = (factorial(n) / (factorial(r)*factorial(n-r)))
    return p**r * (1 - p)**(n - r) * frac

def poisson(r, lam):
    return (lam**r * np.exp(-lam)) / factorial(r)

def poisson_cumulative(r_range, lam, limits=True):
    """Calculates the cumulative probability for a Poisson distribution.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    r_range: array-like
              if limits=True, then this array should contain the upper and lower limit
              of the r values that are desired to be calculated. The r value is the number
              of outcomes.
              Otherwise, this should contain the array of r_values that are desired to be
              summed.
    
    lam: integer
         the expected number of outcomes.
    
    limits: boolean
            default is True.
            True - r_range will be treated as an array containing the lower and upper limits
            respectively of the range of r values to be calculated.
            False - r_range will be treated as an array containing the r values to be
            calculated.
            
    """
    total = 0
    if limits == True:
        if len(r_range) != 2:
            print("When limits=True, r_range should contain simply a lower an upper limit")
            return None
            
        a = r_range[0]
        b = r_range[1]
        r_values = np.linspace(a, b, 1 + (b - a))

        for r in r_values:
            total += poisson(r, lam)
            
    else:
        for r in r_range:
            total += poisson(r, lam)
        
    return total

def gaussian(x, mu, sigma):
    return (1 / (sigma*np.sqrt(2*np.pi))) * np.exp(-((x - mu)**2)/(2*sigma**2))

def Gamma(n):
    return factorial(n-1)

def chi_squared(chisq, nu):
    return (2**(-nu/2) * chisq**(nu/2 - 1) * np.exp(-chisq/2)) / Gamma(nu/2)

def uniform(x, N):
    return 1/N
    

In [2]:
omega = [.5, .9, 1.2, 1.5, 1.8, 2, 3.4, 4.1, 5, 5.1, 7.5, 8.5]
k = [.7, .8, 1.1, 1.2, 1.5, 1.8, 1.9, 2, 2.5, 2.6, 2.9, 3.5]

In [3]:
V = np.array([[variance(omega), covariance(omega, k)],
             [covariance(omega, k), variance(k)]])

eigenvalues, eigenvectors = eig(V)
eigenvectors

array([[ 0.95135429, -0.30809905],
       [ 0.30809905,  0.95135429]])

In [4]:
omega = [1, 2.5, 3, 4, 4.5, 6]
print(np.mean(omega))
print(variance(omega))
print(std(omega))
print(skew(omega))

3.5
3.0
1.7320508075688772
0.0


In [5]:
omega = [.5, 1, 1.5, 1.6, 3, 2.1, 2.5]
print(np.mean(omega))
print(variance(omega))
print(std(omega))
print(skew(omega))

1.7428571428571427
0.7428571428571429
0.8618916073713346
0.026638133469581587


In [6]:
x = [.1, .22, .25, .5, .55, .7, .8, .9, 1, 1.11, 1.12]
y = [1, 1.1, 1.1, 1.2, 1.3, 1.4, 1.4, 1.3, 1.6, 1.5, 1.4]
z = [.1, -.2, .3, .4, .1, -.4, .1, -.1, .6, .7, -.3]

covariance(y,z)

0.017000000000000008

In [7]:
.017 / (std(y) * std(z))

0.25806092613071363

In [8]:
x = [0, .2, .3, .4, .5, .7, .8, .9, 1, .9, 1.1]
y = [.9, 1.1, 1.2, 1.2, 1.3, 1.4, 1.5, 1.3, 1.6, 1.5, 1.3]
z = [-.1, -.2, .1, .2, .1, 0, .2, .1, .5, .6, .3]

covariance(x, y) / (std(x) * std(y))

0.8332164480906891

In [9]:
x = [.5, .7, .8, .9, 1.1, 1.3]
y = [.9, .8, 1.1, 1.2, 1.2, 1]

spearman(x, y)

0.5857142857142856

In [10]:
x = [.1, .3, .2, 0, .4, .5, .1, .2, .6, .5]
y = [.5, .7, .2, .3, .8, .1, .9, 0, .4, .6]

spearman(x, y)

-0.027272727272727337

In [11]:
binomial(3, .4, 5)

0.23040000000000005

In [12]:
poisson(0, 3)

0.049787068367863944

In [13]:
poisson(1, 4), poisson(5, 4)

(0.07326255555493671, 0.15629345185053165)

In [14]:
poisson(1, 4) / poisson(5, 4)

0.46875000000000006

In [15]:
gaussian(1, 0, 1) / gaussian(0, 0, 1)

0.6065306597126334

## Chi-squared

In [16]:
def chi_squared(chisq, nu):
    return (2**(-nu/2) * chisq**(nu/2 - 1) * np.exp(-chisq/2)) / Gamma(nu/2)

In [17]:
chi_squared(5,4)

0.1026062482798735

In [18]:
chi_squared(1, 10)

0.0007897534631674915

In [19]:
chi_squared(2, 8)

0.030656620097620196

In [20]:
stats.chi2.sf(8, 2)

0.018315638888734182

# Uncertainties

## Weighted Averages

In [21]:
def weighted_av_single(x, errors):
    """Calculates the weighted average of a set of measurements for a 
    single observation.
    
    Returns the weighted average and the error.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    x: list containing the measurements
    errors: list containing the errors.
    """
    
    total_top = 0
    total_bottom = 0
    
    for i in range(len(x)):
        var = errors[i]**2
        total_top += x[i] / var
        total_bottom += 1/var
        
    mean_x = total_top/total_bottom
    sigma_x = 1/np.sqrt(total_bottom)
        
    return mean_x, sigma_x

In [22]:
weighted_av_single([.655, .59, .789], [.024, .08, .071])

(0.6628564627744128, 0.021870089601599807)

In [23]:
weighted_av_single([1, 2], [.5, .5])

(1.5, 0.35355339059327373)

In [24]:
covariance([2, 1.5], [0, -1])

0.25

In [25]:
def weighted_av_set(measurements, errors, correlation, precision=None):
    """Calculates the weighted average of a set of measurements for a 
    set of observables.
    
    Returns the weighted average and the error.
    
    CURRENTLY ONLY BUILT FOR 2D.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    measurements: list of lists containing the measurements for each
                    experiment
    errors: list of lists containing the errors for each experiment
    correlation: list of lists containing the correlations for each 
                    experiment
    """
    V_list = []
    x_list = []
    size = len(measurements)
    
    for i in range(size):
        corr_mat = np.ones((size, size))
        cov_mat = np.ones((size, size))*correlation[i][0]*errors[i][0]*errors[i][1]
        for j in range(len(errors[i])):
            cov_mat[j, j] = errors[i][j]**2
            
            #corr_mat[j, -j-1] = correlation[i][0]

        inv_cov = np.linalg.inv(cov_mat)
        
        V_list.append(inv_cov)
        x_list.append(inv_cov @ measurements[i])
            
    V = np.linalg.inv(sum(V_list))
    x = V @ (sum(x_list))
    
    if precision == None:
        return x, V
    
    else:
        return np.round(x, precision), np.round(V, precision)
            
            


# format: [expt1], [expt2], ...
measurements = [[2, 0], [1.5, -1]]
errors = [[.2, .5], [.5, .3]]
correlation = [[.5], [.1]]

weighted_av_set(measurements, errors, correlation)

(array([ 1.81907285, -0.73774834]),
 array([[0.02930861, 0.01299338],
        [0.01299338, 0.06615894]]))

In [26]:
# format: [expt1], [expt2], ...
measurements = [[2, 0], [1.5, .5]]
errors = [[.5, .1], [.5, .2]]
correlation = [[0], [.3]]

weighted_av_set(measurements, errors, correlation)

(array([1.58506224, 0.10995851]),
 array([[0.12033195, 0.00311203],
        [0.00311203, 0.00792531]]))

In [27]:
A = [3, 2]
B = [0, 1]
C = [0, 1]
weighted_av_single(B, [1, 1])

(0.5, 0.7071067811865475)

In [28]:
def hypothesis_ratio(data, hypotheses=[gaussian, uniform], params=[[0, 1], [10]]):
    """Calculates the product of all ratios between the two hypotheses
    for a given set of data.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    data: the data to which the two hypotheses apply
    hypotheses: the two distributions that we wish to compare for the data
    params: the parameters for the given hypotheses"""
    h0, h1 = hypotheses
    PI = 1
    for w in data:
        PI *= h0(w, *params[0]) / h1(w, *params[1])
    
    return PI

In [29]:
omega = [-1.0, -0.9, -0.7, -0.1, 0.0, 0.1, 0.2, 0.5, 0.6, 1.0]
hypothesis_ratio(omega)

140289.8050224156

In [30]:
omega = {-4, -3, -2, -1, 0, 1, 2, 3, 4, 5}
hypothesis_ratio(omega)

3.5611082650219095e-13

In [31]:
omega = {0, .1, .15, .2, .21}
hypothesis_ratio(omega, hypotheses=[uniform, gaussian], params=[[len(omega)], (.15, .9)])

0.019061171028699187

In [32]:
0.05 / 100

0.0005

In [33]:
def bayes_theorem(p_c_a, p_c_b, p_a):
    """
    Calculates Bayes' Theorem for a scenario with two possible main
    outcomes.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    p_c_a: probability of event c given a is true
    p_c_b: probability of event c given b is true
    p_a: probability of event a being true
    
    -+-+-+-+-+-
    EXAMPLE
    -+-+-+-+-+-
    
    Consider a test that correctly predicts infection at a rate of 98%,
    and incorrectly predicts infection at a rate of 0.05%. The percentage
    of infected individuals in the population is 0.01%. The probability
    that someone is infected given a positive test result can be found
    using the following logic:
    
    - Event a is infection
    - Event b is health
    - Event c is a positive test
    
    bayes_theorem(0.98, 0.0005, 0.0001)
    
    """
    
    p_b = 1 - p_a
    p_c = p_c_a*p_a + p_c_b*p_b
    
    p_a_c = (p_c_a*p_a) / (p_c)
    
    return p_a_c

In [34]:
bayes_theorem(.98, .0005, .0001)

0.1638933021155615

In [35]:
bayes_theorem(.9999, .0001, .001)

0.9091653027823241

In [36]:
bayes_theorem(.9999, 1/1_000_000, .1)

0.9999909991809255

In [37]:
omega = [0, 1, 2, 4, 6]
hypothesis_ratio(omega, [poisson, poisson], [[3], [4]])

3.525890604554078

In [38]:
omega = [1, 2, 3, 5, 7]
hypothesis_ratio(omega, [poisson, poisson], [[3], [4]])

0.8367103680728915

In [39]:
omega = [1, 2, 3, 4, 5]
hypothesis_ratio(omega, [binomial, gaussian], [[.4, len(omega)], [3, 1]])

0.23838255092637145

In [40]:
1 - poisson_cumulative([0, 14], 5)

0.0002262536761766798

# Fitting

In [41]:
def chisq_scan(measurements, errors, scan_range, increment=.1,
              same_errors=False):
    """Estimates the parameters that minimize chi-squared using
    scanning across a desired range.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    measurements: array
                  list of measurements
    
    errors: array
            list of errors. If all errors are equal, can simply use
            a list of length 1 with the error, and set same_errors=True.
            
    scan_range: tuple
                the range across which you want parameters to be searched.
                
    increment: float 
               the increments between consecutive searches.
               
    same_errors: boolean
                 default is False. Set to true if all errors are equal,
                 and allow errors to only contain one element with the
                 error.
                 
    -+-+-+-+-+-
    RETURNS
    -+-+-+-+-+-
    
    x_min: the bin that corresponds to the lowest total chi-squared value
    
    sigma: the error associated to x_min
    
    chi_totals: the chi-squared total values for each bin
                 
    """
    a, b = scan_range
    x_values = np.arange(a, b+increment, increment)
    N = len(measurements)
    
    chi_totals = []
    
    if same_errors == True:
        errors = [errors[0] for i in range(N)]
    
    for x in x_values:
        chi_values = []
        for i in range(len(measurements)):
            chi = ((measurements[i] - x) / errors[i])**2
            chi_values.append(chi)
        chi_totals.append(sum(chi_values))
        
    idx_min = np.argmin(chi_totals)
    delta_chi = 0
    idx_low = idx_min
    while delta_chi < 1:
        idx_low -= 1
        delta_chi = chi_totals[idx_low] - chi_totals[idx_min]

    delta_chi = 0
    idx_high = idx_min
    while delta_chi < 1:
        idx_high += 1
        delta_chi = chi_totals[idx_high] - chi_totals[idx_min]
    
    x_min = x_values[idx_min]
    chi_errors = []
    for i in range(-1, 2, 2):
        delta_chi = 0
        idx = idx_min
        while delta_chi < 1:
            idx += i
            delta_chi = chi_totals[idx] - chi_totals[idx_min]
        chi_errors.append(abs(x_min - x_values[idx]))
    
    sigma = np.mean(chi_errors)
    return x_min, sigma, chi_totals

In [42]:
measurements = [1.2, 1.8]
chisq_scan(measurements, [.3], (1, 2), same_errors=True)[0]

1.5000000000000004

In [43]:
measurements = [.662, .625, .897, .614, .925, .694, .601]
errors = [.039, .091, .100, .160, .160, .061, .239]
chisq_scan(measurements, errors, (.6, 1), increment=.01)[0]

0.6900000000000001

In [44]:
weighted_av_single(measurements, errors)

(0.6901364773080294, 0.0283683980386136)

In [45]:
measurements_7 = [1.3, 1.1]
errors = [.1, .2]
chisq_scan(measurements_7, errors, scan_range=(1.1, 1.4),increment=.05)[0]

1.2500000000000002

In [46]:
weighted_av_single(measurements_7, errors)

(1.26, 0.0894427190999916)

In [47]:
def lin_least_squares_2d(x, y, sig):
    """
    Estimates the parameters a and b for a 2d dataset for a model following
    y = ax + b.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    x: array-like
        contains x values
        
    y: array-like
        contains y values
        
    sig: float
         error associated with the values
    """
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    x2_mean = np.mean([i**2 for i in x])
    xy_mean = np.mean([i*j for i, j in zip(x, y)])
    
    N = len(x)
    
    def a_func(xy_mean, x_mean, y_mean, x2_mean):
        return (xy_mean - x_mean*y_mean)/(x2_mean - x_mean**2)
    
    def b_func(x_mean, y_mean, a):
        return y_mean - a*x_mean

    def sig_a_func(sig, N, x_mean, x2_mean):
        return np.sqrt((sig**2)/(N*(x2_mean - x_mean**2)))
    
    def sig_b_func(sig, N, x_mean, x2_mean):
        return np.sqrt((sig**2 * x2_mean)/(N*(x2_mean - x_mean**2)))
    
    a = a_func(xy_mean, x_mean, y_mean, x2_mean)
    b = b_func(x_mean, y_mean, a)
    sig_a = sig_a_func(sig, N, x_mean, x2_mean)
    sig_b = sig_b_func(sig, N, x_mean, x2_mean)
    
    return a, b, sig_a, sig_b

In [48]:
x = [1.1, 1.5, 2, 3.1, 4.2, 5]
y = [2, 2.9, 4.2, 6, 8, 10]
lin_least_squares_2d(x, y, .1)

(1.9753613214039927,
 -0.0472677219545794,
 0.028738084397507524,
 0.09065791491167298)

In [49]:
measurements = [.655, .59, .789]
errors = [.024, .08, .071]
chisq_scan(measurements, errors, (.6, .7), increment=.01)

(0.66,
 0.025000000000000022,
 [12.353455140073633,
  9.934205142828816,
  7.933102035530877,
  6.3501458181798185,
  5.185336490775643,
  4.438674053318347,
  4.110158505807933,
  4.199789848244398,
  4.707568080627745,
  5.633493202957973,
  6.977565215235081])

In [50]:
measurements = [1.3, 1.1]
errors = [.1, .2]
x_mean, sig, chisq = chisq_scan(measurements, errors, (1.1, 1.4),
                                increment=.05)
print(u"<x> = {:.2f} \u00B1 {:.2f}".format(x_mean, sig))

<x> = 1.25 ± 0.10


# MVA

In [51]:
def fisher_disc(mu_a, mu_b, V_a, V_b):
    """Calculates the fisher discriminant coefficients for a dataset
    in which we know which elements are of class A and which are of class
    B, using the mean and variance of each class.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    mu_a / mu_b: matrix with the mean for class A / B
    
    V_a / V_b: matrix with variance for class A / B
    """
    delta_mu = mu_a - mu_b
    W = V_a + V_b
    W_inv = np.linalg.inv(W)
    return W_inv @ delta_mu

In [52]:
mu_a = np.vstack([1, 2])
mu_b = np.vstack([0, 1])
V_a = np.array([[1, 2], [2, 2]])
V_b = np.array([[1, 1], [1, 2]])

fisher_disc(mu_a, mu_b, V_a, V_b)

array([[-1.],
       [ 1.]])

In [53]:
mu_a = np.vstack([1, .7, .5])
mu_b = np.vstack([.6, 1, .2])
V_a = np.array([[.2, .1, 0], [.1, .1, 0], [0, 0, .2]])
V_b = np.array([[.15, 0, .1], [0, .3, 0], [.1, 0, .3]])

fisher_disc(mu_a, mu_b, V_a, V_b)

array([[ 1.36065574],
       [-1.09016393],
       [ 0.32786885]])

In [54]:
ratio_vals = np.arange(0, 1.1, .1)
for i in ratio_vals:
    print(gaussian(i, .5, .2))

0.08764150246784272
0.2699548325659403
0.6475879783294588
1.2098536225957173
1.7603266338214976
1.9947114020071635
1.7603266338214971
1.2098536225957166
0.6475879783294587
0.2699548325659403
0.08764150246784272


In [55]:
def bayes_classifier_NEW(xs, ys=None, distributions_x=[gaussian, uniform],
                        distributions_y=None, params_x=[[0, 1], [10]],
                        params_y=None, categories=None):
    
    """Creates a Bayesian classifier to categorise events in a dataset.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    xs: array-like. 
        the x data that we want to classify, or simply the data for a 
        one dimensional dataset.
        
    ys: array-like.
        the y data for a 2d dataset.
    
    distributions_x: list.
                   contains the functions that act as the distributions
                   for each classifier model for the x data (or simply
                   the data if ys=None).
                   
    distributions_y: list.
                    contains the functions that act as the distributions
                    for each classifier model for the y data.
                   
    params_x / params_y: list of lists.
            containing the parameters for the distribution functions for
            the x / y data
            
    categories: default=None.
                otherwise should be a list containing the class names
                as strings
                
    -+-+-+-+-+-
    RETURNS
    -+-+-+-+-+-
    class_list: list containing:
                - indices of the classified events corresponding to
                the distribution in the distributions list
                - names of the classified class if categories != None.
    """
    
    class_list = []
    p_list = np.zeros((len(xs), len(distributions_x)))
    if ys == None:
        for row, x in enumerate(xs):
            for i in range(len(distributions_x)):
                p_list[row, i] = (distributions_x[i](x, *params_x[i]))

    
    else:
        for row, (x, y) in enumerate(zip(xs, ys)):

            for i in range(len(distributions_x)):
                p_a_x = distributions_x[i](x, *params_x[i])
                p_a_y = distributions_y[i](y, *params_y[i])
                p_list[row, i] = (p_a_x*p_a_y)
            
    for data_point in p_list:    
        classification = np.argmax(data_point)
        if categories != None:
            classification = categories[classification]
        class_list.append(classification)
        
    return class_list

In [56]:
x = [-1, 0, 1, 3, 2, .5]
y = [1, 2, 1, 3, -1, 1]
classes = ["A", " B"]

bayes_classifier_NEW(x, y, [gaussian, gaussian], [gaussian, gaussian], 
                   [[0, 1], [1, 1]], [[0, 1], [2, 1]], categories=classes)

['A', ' B', ' B', ' B', 'A', 'A']

In [57]:
from PyStats import bayes_classifier

x = [-1, 0, 1, 3, 2, .5]
y = [1, 2, 1, 3, -1, 1]
classes = ["A", " B"]

bayes_classifier(x, y, [gaussian, gaussian], [gaussian, gaussian], 
                   [[0, 1], [1, 1]], [[0, 1], [2, 1]], categories=classes)

['A', ' B', ' B', ' B', 'A', 'A']

In [62]:
def separation(mu_a, mu_b, sig_a, sig_b):
    """
    Calculates the separation between two distributions of events.
    
    -+-+-+-+-+-
    PARAMETERS
    -+-+-+-+-+-
    
    mu_a / mu_b: mean of the distribution a / b
    
    sig_a / sig_b: error of distribution a / b
    """
    
    return (mu_a - mu_b)**2 / (sig_a**2 + sig_b**2)